In [0]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn import svm

# Mounting Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Reading files from directory and creating dataframe 

In [0]:
import sys
import glob
import errno
import ntpath

df=[]
path = '/content/drive/My Drive/dataset/*.txt'   
files = glob.glob(path)   
for name in files:
        with open(name,'rb') as f:
          temp=[]
          temp.append(f.read())
          filename=ntpath.basename(name)
          splitlist=filename.split('_')
          num=splitlist[1].split('.')
          temp.append(num[0])
          # temp.append(filename)
          df.append(temp)


# Making of CSV file

In [5]:
df = pd.DataFrame(df, columns = ['text', 'clusternumber']) 
print(df)
df.to_csv(r'Q6.csv', index = False)

Empty DataFrame
Columns: [text, clusternumber]
Index: []


In [0]:
df=pd.read_csv("/content/drive/My Drive/Q6.csv")

# Preprocessing of dataframe, Removing punctuations, in lowecase, e.t.c

In [8]:
print(df)
def clean_text(text):
  text=text.lower()
  text=re.sub('\[.*"\']','',text)
  text=text.replace("\n",' ')
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  return text

round1= lambda x: clean_text(str(x))

                                                   text  clusternumber
0     b"Incredibles win animation awards\n\nThe Incr...              2
1     b'Blair \'up for it\' ahead of poll\n\nTony Bl...              3
2     b'Call centre users \'lose patience\'\n\nCusto...              1
3     b'Kennedy predicts bigger turnout\n\nVoters\' ...              3
4     b'Blair to face trust issue head on\n\nTony Bl...              3
...                                                 ...            ...
1720  b'$1m payoff for former Shell boss\n\nShell is...              1
1721  b'UKIP outspent Labour on EU poll\n\nThe UK In...              3
1722  b'Savvy searchers fail to spot ads\n\nInternet...              5
1723  b'Poll explains free-kick decision\n\nReferee ...              4
1724  b'Critics back Aviator for Oscars\n\nMartin Sc...              2

[1725 rows x 2 columns]


In [9]:
actual_labels=df.clusternumber
print(actual_labels)
clean_data=pd.DataFrame(df.text.apply(round1))
print(clean_data)

0       2
1       3
2       1
3       3
4       3
       ..
1720    1
1721    3
1722    5
1723    4
1724    2
Name: clusternumber, Length: 1725, dtype: int64
                                                   text
0     bincredibles win animation awardsnnthe incredi...
1     bblair up for it ahead of pollnntony blair say...
2     bcall centre users lose patiencenncustomers tr...
3     bkennedy predicts bigger turnoutnnvoters pent ...
4     bblair to face trust issue head onnntony blair...
...                                                 ...
1720  b1m payoff for former shell bossnnshell is to ...
1721  bukip outspent labour on eu pollnnthe uk indep...
1722  bsavvy searchers fail to spot adsnninternet se...
1723  bpoll explains freekick decisionnnreferee grah...
1724  bcritics back aviator for oscarsnnmartin scors...

[1725 rows x 1 columns]


# NLP processing using Tfidfvectorizer

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv1=TfidfVectorizer(stop_words='english')
data_cv1=cv1.fit_transform(clean_data.text)
pp_df1=pd.DataFrame(data_cv1.toarray(),columns=cv1.get_feature_names())
pp_df1=np.array(pp_df1)
print(pp_df1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
import copy
import random

# Selecting K centroids

In [12]:
rsam=[[]]*5
for i in range(0,5):
  rsam[i]=pp_df1[random.randrange(0,1725)]
  print(rsam[i])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


# Applying k-means algorithm(unsupervised Learning)

In [13]:


# print(y_train1)
error=1

while(error>0.01):
  predlist=[]
  new_list=[]
  for i in pp_df1[:,:]:
    result=[]
    index=0
    for j in rsam:
      eudist=0.0
      val=np.linalg.norm(i-j)
      result.append((val,index+1))
      index+=1
    # print(result)
    # print(min(result))
    res=min(result)
    predlist.append(res)
    new_list.append(res[1])
  print(predlist)
  mp1=[[0]*39278]*5
  count=[0]*5
  s=pp_df1
  # print(X_train.shape)
  for k in range(0,len(predlist)):
    mp1[predlist[k][1]-1]+=s[k]
    count[predlist[k][1]-1]+=1
    # m[i]=np.mean(X_train1[])
    # mp[k-1][1]+=1
  # print(mp1[1][0])
  for i in range(0,5):
    mp1[i]/=count[i]
  # print(mp1[1][0])
  error=0
  mp1=np.array(mp1)
  # rsam=np.array(rsam)
  # print(mp1.shape)
  # print(rsam.shape)
  for o in range(0,5):
    error=error+np.linalg.norm(mp1[o]-rsam[o])
  rsam=copy.deepcopy(mp1)
  print(error)

[(1.3977113033635602, 5), (1.331178320965633, 1), (1.380552200553406, 4), (1.3088142612259366, 1), (1.3572303001444688, 1), (1.393434281883039, 3), (1.3771749575554586, 1), (1.3979219830878873, 2), (1.3993741220283942, 3), (1.3888227490924938, 1), (1.3831998138018697, 1), (1.388256627474716, 5), (1.2937588091041852, 5), (1.3860330293296288, 3), (1.384495959568244, 1), (1.3747014199982757, 4), (1.3873841726559966, 5), (1.390629783169864, 4), (1.389003047929014, 5), (1.3983627027713517, 2), (1.4019972297620624, 5), (1.37495246390775, 1), (1.389035704568892, 3), (1.4053499323892706, 1), (1.3857974083061597, 3), (1.3819462121842545, 3), (1.4030106891395493, 1), (1.3962601110768507, 3), (1.3517506469062222, 1), (1.3753926703485078, 3), (1.3798660304239836, 1), (1.389836251002267, 1), (1.396439999084542, 3), (1.3802925541696351, 5), (1.3735490082290889, 5), (1.38613447231845, 3), (1.3876506049524948, 1), (1.3364186016420185, 5), (1.394287776448417, 2), (1.3947075117196286, 3), (1.36570999102

In [14]:
print(len(predlist))

1725


# homogeneity_score

In [1]:
from sklearn.metrics.cluster import homogeneity_score
reasult=homogeneity_score(actual_labels,new_list)
print(reasult)

 0.856231884057971
